In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


running k-means on cuda:0..


[running kmeans]: 7it [00:00, 34.47it/s, center_shift=0.000046, iteration=7, tol=0.000100]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

# TODO: auto sim cutoff needed
decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=2)
decomp.load()

Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Trying 50 clusters for layer blocks.0.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 84it [00:29,  2.81it/s, center_shift=0.000000, iteration=84, tol=0.000100]  


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Trying 50 clusters for layer blocks.1.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 77it [00:28,  2.70it/s, center_shift=0.000000, iteration=77, tol=0.000100]  


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Trying 50 clusters for layer blocks.2.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


...

CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.4.hook_resid_post
Trying 50 clusters for layer blocks.4.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 97it [00:36,  2.65it/s, center_shift=0.000000, iteration=97, tol=0.000100]   


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.5.hook_resid_post
Trying 50 clusters for layer blocks.5.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 40it [00:14,  2.67it/s, center_shift=0.000000, iteration=40, tol=0.000100]   


CLUSTER SHAPE torch.Size([50, 512])
Getting cluster scores for lattice
Getting scores for layers 0 to 1
Getting scores for layers 1 to 2
Getting scores for layers 2 to 3
Getting scores for layers 3 to 4
Getting scores for layers 4 to 5
CLUSTER SCORES [array([[3.49755864, 4.09916179, 4.38130361, ..., 3.61500923, 4.15072528,
        4.40484566],
       [3.92474343, 4.65688239, 5.11339251, ..., 4.11866036, 4.78939758,
        5.10040761],
       [3.8695757 , 4.57967225, 4.99839566, ..., 4.04940191, 4.69246571,
        4.99108061],
       ...,
       [3.65260714, 4.33045255, 4.77294097, ..., 3.83886386, 4.4593576 ,
        4.74419257],
       [3.52389   , 4.17564074, 4.5127983 , ..., 3.68824032, 4.24417722,
        4.52501681],
       [3.47816673, 4.11734485, 4.50113336, ..., 3.64116984, 4.22118926,
        4.49333164]]), array([[0.70333037, 3.92304845, 0.87358022, ..., 3.83998436, 3.70563864,
        3.76602658],
       [0.81791344, 4.6243185 , 0.91250215, ..., 4.54167395, 4.38050541,
   

In [3]:
decomp.lattice_scores[0].shape

(50, 50)

In [4]:
from typing import List
import utils
importlib.reload(utils)
modified_lattice = decomp.lattice_scores.copy()

# TODO: have function which does something like fixing from the a specific neuron?
# Ie have the sub-lattice with only **related** neurons

def cutoff_lattice(lattice: List[List[List[float]]], related_cutoff = 1):
    print(lattice[0].sum())
    r = [(layer > related_cutoff) * layer for layer in lattice]
    print(r[0].sum())
    return r

def create_sublattice(lattice, layer: int, idx_in_layer: int, related_cutoff = 10):
	with_cutoff = cutoff_lattice(lattice, related_cutoff)

LAST_FEAT = 20
modified_lattice = cutoff_lattice(decomp.lattice_scores, related_cutoff=15)

# utils.top_k_dag_paths_dynamic(modified_lattice, k=10_000)

10436.00001455494
200.5030490747813


In [5]:
# TODO:
""""
We need to combine the 
"""

import numpy as np
from utils import get_random_cutoff

score_path = [8, 57, 89]
score_path = [10, 63, 89]
score_path = [3, 3, 1]
# score_path = [39, 14, 2, 3, 4]
score_path = [1, 1, 2, 3, 4, 5]
"""
Paths which seem to have meaning

score_path = [1, 1, 2, 3, 4]
Very strong for Q: . Specifically on the space
"""

to_score = [get_random_cutoff(d, 100) for d in ds][:300]
scores = decomp.score(
    to_score,
    score_path=score_path,
    weighting_per_layer=np.array([1, 1, 1, 1, 1, 1])
)


DS (6, 5412, 512) 5412
(5412, 6, 512) [1, 1, 2, 3, 4, 5]


In [6]:
from IPython.core.display import display, HTML
import math
import numpy as np

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!
act_simp = scores_reord
act_simp = [[[[0 if math.isnan(tok * 100) else int(math.exp(tok * 100))]] for tok in s] for s in scores_reord]
# TODO sep fun
from circuitsvis.utils.render import RenderedHTML, render
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)
display(HTML(str(html)))

/tmp/ipykernel_8451/3908721336.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Isolate Specific Neurons

In [57]:
import cluster_model
importlib.reload(cluster_model)

LAYER = 3
NODE = 10
# TODO: this ain't right!! We need specific paths!!
ret = cluster_model.restrict_to_related_path(decomp.lattice_scores, LAYER, NODE, n_paths=1000)
len(ret), ret

(1000,
 [([35, 16, 10, 42, 10, 22], 437.741979506146),
  ([35, 16, 10, 42, 10, 44], 437.72526742508035),
  ([35, 16, 10, 42, 10, 31], 437.7144532736593),
  ([35, 16, 10, 42, 10, 11], 437.6851023377412),
  ([35, 16, 10, 42, 10, 6], 437.63453430761274),
  ([35, 16, 10, 42, 10, 47], 437.62611857515276),
  ([35, 16, 10, 42, 10, 4], 437.6225714607599),
  ([35, 16, 10, 42, 10, 10], 437.5950445767985),
  ([35, 16, 10, 42, 10, 13], 437.5709182643941),
  ([35, 16, 10, 42, 10, 7], 437.55283322368905),
  ([35, 16, 10, 42, 10, 9], 437.5439176097161),
  ([35, 16, 10, 42, 10, 2], 437.541618356644),
  ([35, 16, 10, 42, 10, 26], 437.5364211716251),
  ([35, 16, 10, 42, 10, 38], 437.4813078657053),
  ([35, 16, 10, 42, 10, 42], 437.4765155091036),
  ([35, 16, 10, 42, 10, 3], 437.4718620546146),
  ([35, 16, 10, 42, 10, 20], 437.4557457278743),
  ([35, 16, 10, 42, 10, 32], 437.398644814969),
  ([35, 16, 10, 42, 10, 16], 437.34923265678935),
  ([35, 16, 10, 42, 10, 27], 437.3325155019438),
  ([35, 16, 10, 4

In [58]:
score_path = [35, 16, 10, 42, 10, 22]

"""
Paths which seem to have meaning

score_path = [1, 1, 2, 3, 4]
Very strong for Q: . Specifically on the space

score_path_path = [3, 15, 19, 11, 12]
Very strong for Q: . when context is about *programming*
"""

to_score = [d[:100] for d in ds][:600]
scores = decomp.score(
    to_score,
    score_path=score_path,
    weighting_per_layer=np.array([1, 1, 1, 1, 1, 1]) # TODO: how do we weight???
)

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!
act_simp = scores_reord
act_simp = [[[[0 if math.isnan(tok * 100) else int(math.exp(tok * 100))]] for tok in s] for s in scores_reord]
# TODO sep fun
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)
display(HTML(str(html)))

DS (6, 5500, 512) 5500
(5500, 6, 512) [35, 16, 10, 42, 10, 22]
